<a href="https://colab.research.google.com/github/Faabry/Data_Analysis/blob/main/Falecimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> <b> Web Scraping Funerário </b> </h1>
<p> Web scraping, também conhecido como raspagem de dados, é o processo de extrair informações e dados estruturados de sites da web. Isso é feito usando software automatizado, que lê o conteúdo do site e extrai as informações desejadas em um formato estruturado, como um arquivo CSV ou um banco de dados. Essas informações podem ser usadas para análise de dados, pesquisa de mercado, monitoramento de preços, entre outras finalidades. No entanto, é importante lembrar que alguns sites podem proibir ou restringir o uso de técnicas de web scraping.</p>

In [ ]:
from platform import python_version
print(f"Versão Python para esse projeto: {python_version()}")

Versão Python para esse projeto: 3.9.16


In [ ]:
!pip install -q watermark

<h1> <b> Importando as bibliotecas necessárias para a execução do projeto </b> </h1>

In [ ]:
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [ ]:
%reload_ext watermark

%watermark -a "Airton Fabre" --iversions

Author: Airton Fabre

numpy   : 1.22.4
requests: 2.27.1
pandas  : 1.4.4



<h1> <b> Raspagem de dados </b> </h1>

In [ ]:
# Definindo o ano através da biblioteca datetime para que o resultado seja
# 100% dinâmico em relação ao presente momento da execução do script
now = datetime.now()
year = now.year

url = "https://www.araras.sp.gov.br/falecimentos/pesquisa_falecimentos.php"

# Esse parâmetro é usado para que o usuário diga ao servidor que não é uma 
# máquina fazendo essa busca
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"}


# Criando um dicionário para armazenar as informações da página web
falecidos = {"Nome":     [],
             "Idade":    [],
             "DataFal":  [],
             "DataSep":  []}

# Criando um looping for para extrair todos os dados em um periodo de tempo

# for y in range(2012, year + 1): '''(na primeira execução foram coletados todos os registros)'''
  
# Coletar somente os dados a partir do presente ano, pois não há necessidade de pegar de todos os anos uma segunda vez
for m in range(1, 13):
  data = {"mes": {m}, "ano": {year}}
  response = requests.post(url, data=data, headers=headers)
  soup = BeautifulSoup(response.text, "html.parser", from_encoding="utf-8")
  feretros = soup.find_all("div", {"class":"col-md-9 sub-falecimento"})
  data_fal = soup.find_all("div", {"class":"col-md-7 sub-falecimento"})
  result = [i.get_text() for i in feretros]
  data_falecimento = [i.get_text() for i in data_fal]

  for k, v in enumerate(result):
      if k % 2 == 0:
          falecidos["Nome"].append(v)
      else:
          falecidos["Idade"].append(v)

  for k, v in enumerate(data_falecimento):
      if k % 2 == 0:
          falecidos["DataFal"].append(v)
      else:
          falecidos["DataSep"].append(v)

<p>Foi raspado todos os dados desde o primeiro registro em 2012 até o momento atual</p>

<h1> <b> Criando um DataFrame </b> </h1>
<p> Nosso DataFrame será criado a partir do dicionário com os dados extraídos do site </p>

In [ ]:
# Primeira execução do script com dados a partir de 2012.
# O nome do dicionário foi alterado para que não houvesse divergencia e nem 
# a necessidade de escrever um novo script
df = pd.DataFrame(falecimentos)
df.head()

,Nome,Idade,DataFal,DataSep
0,Gilson de Jesus Tavares,"Aos 40 anos, Casado. Residia no endereço: Rua:...",31/01/2012,31/01/2012 às - Cemitério Municipal: Sepultam...
1,Luzia de Lourdes Silveira Dias,"Aos 66 anos, Viúva. Residia no endereço: Rua -...",31/01/2012,01/02/2012 às - Cemitério Municipal - Sepulta...
2,Luci Terezinha Martim De Paula,"Aos 53 anos, Casada. Residia no endereço: Rua ...",30/01/2012,30/01/2012 às - Cemitério Municipal - Sepulta...
3,Valter Sorrantino,"Aos 47 anos, Casado. Residia no endereço: Rua ...",30/01/2012,31/01/2012 às - Cemitério Municipal - Sepulta...
4,Antonio Vieira dos Santos,"Aos 55 anos, Solteiro. Residia no endereço: Ru...",30/01/2012,31/01/2012 às - Cemitério Municipal: Sepultam...


In [ ]:
df.shape

(10063, 4)

In [ ]:
# Salvando o dataframe a partir de 2012.
df.to_excel("falecimentos.xlsx", index=False)

In [ ]:
# Segunda execução do script com dados a partir de Jan 2023
df_atual = pd.DataFrame(falecidos)
df_atual.head()

,Nome,Idade,DataFal,DataSep
0,José do Nascimento,"Aos 74 anos, Divorciado. Residia no endereço: ...",31/01/2023,01/02/2023 às 13:30 - Cemitério Municipal de A...
1,Pedro Alves dos Santos,"Aos 70 anos, Casado. Residia no endereço: Aven...",31/01/2023,01/02/2023 às 13:00 - Cemitério de Leme.
2,Clarinda Bueno,"Aos 83 anos, Divorciada. Residia no endereço: ...",30/01/2023,31/01/2023 às 10:30 - Cemitério Municipal de A...
3,JOSELMA ALVES NASCIMENTO,"Aos 48 anos, DIVORCIADA . Residia no endereço:...",30/01/2023,30/01/2023 às 16:00 - CEMITÉRIO MUNICIPAL DE A...
4,Abrahão Vieira,"Aos 79 anos, Divorciado. Residia no endereço: ...",30/01/2023,31/01/2023 às 14:00 - Cemitério dos Eucaliptos.


In [ ]:
df_atual.shape

(238, 4)

<h3><b>Abrindo o DataFrame de 2012 e concatenando ao DataFrame atual</b></h3>

In [ ]:
df = pd.read_excel("falecimentos.xlsx")
df.head()

,Nome,Idade,DataFal,DataSep
0,Gilson de Jesus Tavares,"Aos 40 anos, Casado. Residia no endereço: Rua:...",31/01/2012,31/01/2012 às - Cemitério Municipal: Sepultam...
1,Luzia de Lourdes Silveira Dias,"Aos 66 anos, Viúva. Residia no endereço: Rua -...",31/01/2012,01/02/2012 às - Cemitério Municipal - Sepulta...
2,Luci Terezinha Martim De Paula,"Aos 53 anos, Casada. Residia no endereço: Rua ...",30/01/2012,30/01/2012 às - Cemitério Municipal - Sepulta...
3,Valter Sorrantino,"Aos 47 anos, Casado. Residia no endereço: Rua ...",30/01/2012,31/01/2012 às - Cemitério Municipal - Sepulta...
4,Antonio Vieira dos Santos,"Aos 55 anos, Solteiro. Residia no endereço: Ru...",30/01/2012,31/01/2012 às - Cemitério Municipal: Sepultam...


In [ ]:
df = pd.concat([df, df_atual])
df.head()

,Nome,Idade,DataFal,DataSep
0,Gilson de Jesus Tavares,"Aos 40 anos, Casado. Residia no endereço: Rua:...",31/01/2012,31/01/2012 às - Cemitério Municipal: Sepultam...
1,Luzia de Lourdes Silveira Dias,"Aos 66 anos, Viúva. Residia no endereço: Rua -...",31/01/2012,01/02/2012 às - Cemitério Municipal - Sepulta...
2,Luci Terezinha Martim De Paula,"Aos 53 anos, Casada. Residia no endereço: Rua ...",30/01/2012,30/01/2012 às - Cemitério Municipal - Sepulta...
3,Valter Sorrantino,"Aos 47 anos, Casado. Residia no endereço: Rua ...",30/01/2012,31/01/2012 às - Cemitério Municipal - Sepulta...
4,Antonio Vieira dos Santos,"Aos 55 anos, Solteiro. Residia no endereço: Ru...",30/01/2012,31/01/2012 às - Cemitério Municipal: Sepultam...


<h1><b>Análise Exploratória</b></h1>

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10301 entries, 0 to 237
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Nome     10301 non-null  object
 1   Idade    10301 non-null  object
 2   DataFal  10301 non-null  object
 3   DataSep  10301 non-null  object
dtypes: object(4)
memory usage: 402.4+ KB


In [ ]:
df.shape

(10301, 4)

<p> Não temos nenhum valor ausente, mas os tipos primitivos de algumas colunas como <b> DataFal e DataSep </b> estão incorretos.

<h1> <b> Task 1:</b> Verificar se existem valores duplicados</h1>
<p>O primerio passo é remover possíveis espaços em branco que podem indicar valores diferentes para o pandas</p>

In [ ]:
df = df.apply(lambda x: x.str.strip())

In [ ]:
# Verificando se existe alguma duplicidade nos dados
df[["Nome", "Idade"]].duplicated().sum()

254

In [ ]:
df.loc[df["Nome"] == "Pedro Aparecido Bernardo da Silva"]

,Nome,Idade,DataFal,DataSep
10062,Pedro Aparecido Bernardo da Silva,"Aos 36 anos, . Residia no endereço: Rua Guilhe...",01/04/2023,02/04/2023 às 09:00 - Cemitério Municipal de A...
237,Pedro Aparecido Bernardo da Silva,"Aos 36 anos, . Residia no endereço: Rua Guilhe...",01/04/2023,02/04/2023 às 09:00 - Cemitério Municipal de A...


<p> Como podemos ver temos alguns valores duplicados, o próximo passo é remove-los </p>

In [ ]:
df.drop_duplicates(["Nome", "Idade"], inplace=True)

In [ ]:
df.count()

Nome       10047
Idade      10047
DataFal    10047
DataSep    10047
dtype: int64

In [ ]:
df.head()

,Nome,Idade,DataFal,DataSep
0,Gilson de Jesus Tavares,"Aos 40 anos, Casado. Residia no endereço: Rua:...",31/01/2012,31/01/2012 às - Cemitério Municipal: Sepultam...
1,Luzia de Lourdes Silveira Dias,"Aos 66 anos, Viúva. Residia no endereço: Rua -...",31/01/2012,01/02/2012 às - Cemitério Municipal - Sepulta...
2,Luci Terezinha Martim De Paula,"Aos 53 anos, Casada. Residia no endereço: Rua ...",30/01/2012,30/01/2012 às - Cemitério Municipal - Sepulta...
3,Valter Sorrantino,"Aos 47 anos, Casado. Residia no endereço: Rua ...",30/01/2012,31/01/2012 às - Cemitério Municipal - Sepulta...
4,Antonio Vieira dos Santos,"Aos 55 anos, Solteiro. Residia no endereço: Ru...",30/01/2012,31/01/2012 às - Cemitério Municipal: Sepultam...


<h1> <b>Task 2:</b> Tratar os dados para que se tornem informações</h1>

In [ ]:
pd.set_option("display.max_rows", None)

In [ ]:
# Criando novas colunas para desmembrar os dados
df["Local"] = df["DataSep"].apply(lambda x: x[10::])
df["DataSep"] = df["DataSep"].apply(lambda x: x[0: 11])
df["Idade"] = df["Idade"].apply(lambda x: x[3:7])
df.head()

,Nome,Idade,DataFal,DataSep,Local
0,Gilson de Jesus Tavares,40,31/01/2012,31/01/2012,às - Cemitério Municipal: Sepultamento as 10...
1,Luzia de Lourdes Silveira Dias,66,31/01/2012,01/02/2012,às - Cemitério Municipal - Sepultamento ás 0...
2,Luci Terezinha Martim De Paula,53,30/01/2012,30/01/2012,às - Cemitério Municipal - Sepultamento ás 1...
3,Valter Sorrantino,47,30/01/2012,31/01/2012,às - Cemitério Municipal - Sepultamento ás 0...
4,Antonio Vieira dos Santos,55,30/01/2012,31/01/2012,às - Cemitério Municipal: Sepultamento as 10...


In [ ]:
df["Idade"] = df["Idade"].str.strip()

In [ ]:
df["Idade"].unique()

array(['40', '66', '53', '47', '55', '62', '90', '88', '32', '81', '79',
       '24', '34', '76', '82', '80', '89', '49', '31', '73', '78', '65',
       '96', '74', '42', '7 a', '94', '57', '60', '68', '41', '50', '48',
       '51', '77', '84', '75', '69', '39', '44', '56', '46', '64', '92',
       '21', '52', '43', '72', '59', '58', '67', '54', '61', '87', '63',
       '85', '93', '95', '45', '91', '83', '86', '97', '70', '29', '71',
       '26', '3 a', '33', '35', '36', '27', '22', '19', '38', '28', '37',
       '101', '23', '18', '103', '98', '102', '16', '100', '20', '25',
       '99', '0 a', '14', '17', '12', '105', '30', '104', '4 a', '13',
       '2 a', '15', '8 a', '6 a', '9 a', '5 a', '107', '11', '106'],
      dtype=object)

<p> Existem alguns valores com strings como <b>5 a</b> isso será um problema na análise final e precisa ser resolvido. </p>

In [ ]:
# Substituindo valores "_ a" por números
for c in range(0, 10):
  df["Idade"] = df["Idade"].str.replace(f"{c} a", f"{c}")

In [ ]:
df["Idade"].unique()

array(['40', '66', '53', '47', '55', '62', '90', '88', '32', '81', '79',
       '24', '34', '76', '82', '80', '89', '49', '31', '73', '78', '65',
       '96', '74', '42', '7', '94', '57', '60', '68', '41', '50', '48',
       '51', '77', '84', '75', '69', '39', '44', '56', '46', '64', '92',
       '21', '52', '43', '72', '59', '58', '67', '54', '61', '87', '63',
       '85', '93', '95', '45', '91', '83', '86', '97', '70', '29', '71',
       '26', '3', '33', '35', '36', '27', '22', '19', '38', '28', '37',
       '101', '23', '18', '103', '98', '102', '16', '100', '20', '25',
       '99', '0', '14', '17', '12', '105', '30', '104', '4', '13', '2',
       '15', '8', '6', '9', '5', '107', '11', '106'], dtype=object)

<h1> <b>Task 3:</b> Tratar as colunas de datas e converte-las para o tipo primitivo correto.</b></h1>

In [ ]:
df["DataFal"] = df["DataFal"].str.replace("/", "-")
df["DataSep"] = df["DataSep"].str.replace("/", "-")

<p> Verificando se existem valores estranhos em relação ao ano.</p>

In [ ]:
# Verificando valores unicos em relação à data de sepultamento
df["DataSep"].apply(lambda x: x[6::]).unique()

array(['2012 ', '2011 ', '2023 ', '2013 ', '2014 ', '0123 ', '2015 ',
       '2016 ', '2000 ', '2017 ', '2018 ', '2019 ', '2020 ', '2021 ',
       '2022 ', '2200 '], dtype=object)

<p> Foi observado que existem valores numéricos porém que não correspondem à uma data como por exemplo <b>"0123, 2000, 2200"</b>, será necessário verificar à que ano esses valores estão relacionados. </p>

In [ ]:
df.loc[df["DataSep"].str.contains("2000 ")] #2000 = 2016
df.loc[df["DataSep"].str.contains("2200 ")] #2200 = 2022
df.loc[df["DataSep"].str.contains("0123 ")] #0123 = 2014

,Nome,Idade,DataFal,DataSep,Local
1831,Anna Gonçalves félix,79,12-03-2014,13-03-0123,às 10:00 - Cemitério Municipal.


In [ ]:
df["DataSep"] = df["DataSep"].str.strip()

<h3> <b>Substituindo os valores incorretos para o seu devido ano.</b> </h3>

In [ ]:
df["DataSep"] = df["DataSep"].str.replace("2000", "2016")
df["DataSep"] = df["DataSep"].str.replace("2200", "2022")
df["DataSep"] = df["DataSep"].str.replace("0123", "2014")

In [ ]:
pd.to_datetime(df["DataFal"], format='%d-%m-%Y')
pd.to_datetime(df["DataSep"], format='%d-%m-%Y')

In [ ]:
df["DataSep"].apply(lambda x: x[6::]).unique()

array(['2012', '2011', '2023', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019', '2020', '2021', '2022'], dtype=object)

<h1> <b>Task 4:</b> Verificar se há muita diferença entre a contagem de valores para <b>["DataFal"]</b> em relação à <b>["DataSep"].</b> </h>

In [ ]:
# Contagem de valores para "ano" da coluna "DataSep"
sep = df["DataSep"].apply(lambda x: x[6::]).value_counts()

In [ ]:
# Contagem de valores para "ano" da coluna "DataFal"
fal = df["DataFal"].apply(lambda x: x[6::]).value_counts()

<p> Na coluna <b>["DataFal"]</b> existe um valor de <i>2011</i>, pois a pessoa faleceu no ultimo dia do ano e foi sepultada no primeiro dia do ano seguinte, porém se mantermos esse valor vamos ter problemas, pois para criar um DataFrame é necessário que todas as colunas tenham o mesmo tamanho, então nesse caso como é apenas um valor vamos simplesmete remove-lo.</p>

In [ ]:
a = pd.Series(sep)
b = pd.Series(fal)

# Removendo o valor de 2011
a = a.drop(index="2011")

In [ ]:
# Criando um dicionário para armazenar as informações de cada ano em relação à
# data de falecimento/sepultamento
data_dict = {"sep": a,
             "fal": b}

# Criando um DataFrame para melhor visualização             
result = pd.DataFrame(data_dict)
result["Diferença"] = result["fal"] - result["sep"]
result

,sep,fal,Diferença
2021,1137,1132,-5
2022,1062,1066,4
2020,909,910,1
2017,882,884,2
2019,869,871,2
2015,852,849,-3
2018,850,849,-1
2012,836,836,0
2014,817,820,3
2013,803,804,1


<p> Aparentemente os valores estão normais, uma vez que é completamente normal uma pessoa falecer no ultimo dia do ano e ser sepultada no primeiro dia do ano seguinte. </p>

<h1> <b>Task 5:</b> Ajustar as informações da coluna <b>["Local"]</b>. </h1>

In [ ]:
# Removendo todo e qualquer espaço em branco no inicio ou fim da linha
# em seguida estou colocando todos os caracteres em letra minúscula para que não
# tenhamos problemas com pAlaVRas eScritAS assim 
df["Local"] = df["Local"].str.strip().str.lower()

In [ ]:
df.head()

,Nome,Idade,DataFal,DataSep,Local
0,Gilson de Jesus Tavares,40,31-01-2012,31-01-2012,às - cemitério municipal: sepultamento as 10:...
1,Luzia de Lourdes Silveira Dias,66,31-01-2012,01-02-2012,às - cemitério municipal - sepultamento ás 09...
2,Luci Terezinha Martim De Paula,53,30-01-2012,30-01-2012,às - cemitério municipal - sepultamento ás 15...
3,Valter Sorrantino,47,30-01-2012,31-01-2012,às - cemitério municipal - sepultamento ás 08...
4,Antonio Vieira dos Santos,55,30-01-2012,31-01-2012,às - cemitério municipal: sepultamento as 10:...


<p> Após uma análise foi verificado que existem muitas variações da mesma palavra então será necessário fazer o tratamento de cada uma das variações para que exista apenas um padrão. </p>

In [ ]:
for c in range(7, 10):
  df["Local"] = df["Local"].str.replace(f"às 0{c}:00 - cemitério municipal.", f"0{c}:00 Cemitério Municipal de Araras - SP")
  df["Local"] = df["Local"].str.replace(f"às 0{c}:30 - cemitério municipal.", f"0{c}:30Cemitério Municipal de Araras - SP")
  df["Local"] = df["Local"].str.replace(f"às  - cemitério municipal: sepultamento as 0{c}:00 hrs.", "Cemitério Municipal de Araras - SP")
  
for i in range(10, 18):
  df["Local"] = df["Local"].str.replace(f"às  - cemitério municipal: sepultamento as {i}:00 hrs.", "Cemitério Municipal de Araras - SP")
  df["Local"] = df["Local"].str.replace(f"às  - cemitério Municipal: sepultamento as {i}:30 hrs.", "Cemitério Municipal de Araras - SP")
  df["Local"] = df["Local"].str.replace(f"às  - cemitério Municipal: sepultamento as {i}:00 h.", "Cemitério Municipal de Araras - SP")
  df["Local"] = df["Local"].str.replace(f"às  - cemitério Municipal: sepultamento as {i}:30 h.", "Cemitério Municipal de Araras - SP")
  df["Local"] = df["Local"].str.replace(f"às {i}:00 - cemitério municipal.", f"{i}:00 - Cemitério Municipal de Araras - SP")
  df["Local"] = df["Local"].str.replace(f"às {i}:30 - cemitério municipal.", f"{i}:30 - Cemitério Municipal de Araras - SP")

In [ ]:
df["Local"] = df["Local"].str.replace("cemitério municipal de araras sp.",'Cemitério Municipal de Araras - SP')\
                         .str.replace("às cemitério municipal de araras - sp", "Cemitério Municipal de Araras - SP")\
                         .str.replace("cemitério municipal de araras - spde araras - sp", "Cemitério Municipal de Araras - SP")\
                         .str.replace("às-cemitério municipal.", "Cemitério Municipal de Araras - SP")\
                         .str.replace("às-cemitériomunicipaldeararahoarioadefinirs-sp.", "Cemitério Municipal de Araras - SP")\
                         .str.replace("às-cemitériopq.doseucaliptos-araras-sp.", "Cemitério Pq dos Eucaliptos")\
                         .str.replace("às-crematóriomonumentalemlimeira-sp.", "Crematório Monumental")\
                         .str.replace("às-crematóriopq.daspalmeiras-rioclaro-sp.", "Crematório Pq Das Palmeiras - Rio Claro - SP")\
                      

<ipython-input-400-18b0d580ede4>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Local"] = df["Local"].str.replace("cemitério municipal de araras sp.",'Cemitério Municipal de Araras - SP')\


# Horarios


In [ ]:
# Criando uma nova coluna para receber a informação do horário do sepultamento
df["Horário"] = df["Local"].apply(lambda x: x[0:5]).str.lower()
df.head()

,Nome,Idade,DataFal,DataSep,Local,Horário
0,Gilson de Jesus Tavares,40,31-01-2012,31-01-2012,às - cemitério municipal: sepultamento as 10:...,às -
1,Luzia de Lourdes Silveira Dias,66,31-01-2012,01-02-2012,às - cemitério municipal - sepultamento ás 09...,às -
2,Luci Terezinha Martim De Paula,53,30-01-2012,30-01-2012,às - cemitério municipal - sepultamento ás 15...,às -
3,Valter Sorrantino,47,30-01-2012,31-01-2012,às - cemitério municipal - sepultamento ás 08...,às -
4,Antonio Vieira dos Santos,55,30-01-2012,31-01-2012,às - cemitério municipal: sepultamento as 10:...,às -


In [ ]:
df["Horário"].unique()

array(['às  -', '14:30', 'cemit', '16:00', '16:30', '13:00', '13:30',
       '14:00', 'às 15', '15:30', '10:30', '15:00', '09:30', '10:00',
       'às 00', '08:00', 'às 90', 'às 83', '08:30', 'às 16', '09:00',
       'às 10', 'às 14', 'às 13', '11:00', '17:00', '12:00', 'às 12',
       'às 17', 'às 08', 'às 09', '11:30', 'às 18', '17:30', 'às 8:',
       '12:30', 'às 11', 'às 07', '07:30', 'às 19', 'às 06', 'às 80',
       '07:00', 'às 05'], dtype=object)

In [ ]:
df["Horário"].value_counts()

16:00    1149
10:00    1130
15:00     851
09:00     791
14:00     681
16:30     576
08:00     567
10:30     483
17:00     447
13:00     444
às  -     331
15:30     295
13:30     275
11:00     248
às 16     234
09:30     231
às 10     229
08:30     165
14:30     157
às 15     110
às 13     109
às 09      98
às 14      81
12:00      76
às 08      48
11:30      40
às 00      38
às 17      37
17:30      36
às 11      35
às 12      21
cemit      10
às 18       7
12:30       5
às 19       2
às 90       2
07:00       1
às 80       1
às 06       1
às 83       1
07:30       1
às 07       1
às 8:       1
às 05       1
Name: Horário, dtype: int64

<p> Existem alguns valores não correspondentes à um valor de horas, portanto será necessário tratá-los. </p>

In [ ]:
nan_index = df[(df["Horário"].str.startswith("às  -") & (df["Local"].str[-10:]))].index

In [ ]:
len(nan_index)

331

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
# Imputando o valor "15:30" nos indices corretos
df["Horário"][nan_index] = "15:30"

In [ ]:
df["Horário"].unique()

array(['15:30', '14:30', 'cemit', '16:00', '16:30', '13:00', '13:30',
       '14:00', 'às 15', '10:30', '15:00', '09:30', '10:00', 'às 00',
       '08:00', 'às 90', 'às 83', '08:30', 'às 16', '09:00', 'às 10',
       'às 14', 'às 13', '11:00', '17:00', '12:00', 'às 12', 'às 17',
       'às 08', 'às 09', '11:30', 'às 18', '17:30', 'às  -', 'às 8:',
       '12:30', 'às 11', 'às 07', '07:30', 'às 19', 'às 06', 'às 80',
       '07:00', 'às 05'], dtype=object)

In [ ]:
for c in range(10, 20):
  df["Horário"] = np.where(df["Horário"] == f"às {c}", f"{c}:00", df["Horário"])

In [ ]:
for c in range(6, 10):
  df["Horário"] = np.where(df["Horário"] == f"às 0{c}", f"0{c}:00", df["Horário"])

In [ ]:
df["Horário"] = np.where(df["Horário"] == "cemit", "10:00", df["Horário"])
df["Horário"] = np.where(df["Horário"] == "às 83", "08:30", df["Horário"])
df["Horário"] = np.where(df["Horário"] == "às 00", "10:00", df["Horário"])
df["Horário"] = np.where(df["Horário"] == "às 90", "09:00", df["Horário"])
df["Horário"] = np.where(df["Horário"] == "às 8", "08:00", df["Horário"])
df["Horário"] = np.where(df["Horário"] == "às 80", "08:00", df["Horário"])
df["Horário"] = np.where(df["Horário"] == "às 8:", "08:00", df["Horário"])
df["Horário"] = np.where(df["Horário"] == "às 05", "15:00", df["Horário"])

In [ ]:
df["Horário"].unique()

array(['15:30', '14:30', '10:00', '16:00', '16:30', '13:00', '13:30',
       '14:00', '15:00', '10:30', '09:30', '08:00', '09:00', '08:30',
       '11:00', '17:00', '12:00', '11:30', '18:00', '17:30', 'às  -',
       '12:30', '07:00', '07:30', '19:00', '06:00'], dtype=object)

In [ ]:
df["Horário"].value_counts()

10:00    1400
16:00    1379
15:00     955
09:00     890
14:00     759
15:30     626
08:00     612
16:30     574
13:00     553
10:30     483
17:00     481
11:00     281
13:30     275
09:30     231
08:30     166
14:30     157
12:00      97
11:30      40
17:30      36
às  -      34
18:00       7
12:30       5
07:00       2
19:00       2
07:30       1
06:00       1
Name: Horário, dtype: int64

In [ ]:
np.where(pd.to_datetime(df["Horário"], format='%H:%M', errors='coerce').isna(), '00:00:00', df["Horário"])

array(['15:30', '15:30', '15:30', ..., '16:00', '09:00', '09:00'],
      dtype=object)

In [ ]:
df["Horário"].unique()

array(['15:30', '14:30', '10:00', '16:00', '16:30', '13:00', '13:30',
       '14:00', '15:00', '10:30', '09:30', '08:00', '09:00', '08:30',
       '11:00', '17:00', '12:00', '11:30', '18:00', '17:30', 'às  -',
       '12:30', '07:00', '07:30', '19:00', '06:00'], dtype=object)

# Local

In [ ]:
df["Local"] = df["Local"].apply(lambda x: x[11::])

In [ ]:
df["Local"] = df["Local"].str.lower()

In [ ]:
df["Local"].unique()

array(['ério municipal: sepultamento as 10:30 h.',
       'ério municipal - sepultamento ás 09:30 hrs.',
       'ério municipal - sepultamento ás 15:00 hrs  .',
       'ério municipal - sepultamento ás 08:00 hrs.',
       'ério municipal: sepultamento as 10:00 h.',
       'ério municipal -  sepultamento ás 16:30 hrs.',
       'ério municipal - sepultamento ás 16:00 hrs.',
       'ério municipal: sepultamento as 15:30 h.',
       'ério municipal: sepultamento as 14:00 h.',
       'ério municipal; sepultamento  as 9:00 h.',
       'ério municipal -sepultamento ás 16:30 hrs.',
       'ério municipal - sepultamento ás 10:00.',
       'ério municipal: seputamento as 10:30 h.',
       'ério municipal: sepultamento as 16:00 h.',
       'ério municipal - sepultamento ás 15:30 hrs.',
       'ério municipal: sepultamento as 16:30 h.',
       'ério municipal - sepultamento ás 08:30 hrs.',
       'ério municipal - sepultamento ás 13:30 hrs.',
       'ério municipal - sepultamento ás 14:30 hrs.',
 

In [ ]:
araras = df["Local"].loc[df["Local"].str.contains("araras")].index
araras2 = df["Local"].loc[df["Local"].str.contains("ério municipal")].index

In [ ]:
df["Local"][araras] = "cem. mun de araras sp"
df["Local"][araras2] = "cem. mun de araras sp"

In [ ]:
pq = df["Local"].loc[df["Local"].str.contains("eucalipt")].index
pq2 = df["Local"].loc[df["Local"].str.contains("eucalíp")].index

df["Local"][pq] = "pq dos eucaliptos araras sp"
df["Local"][pq2] = "pq dos eucaliptos araras sp"

In [ ]:
leme = df["Local"].loc[df["Local"].str.contains("leme")].index

df["Local"][leme] = "cem. mun de leme sp"

In [ ]:
monumental = df["Local"].loc[df["Local"].str.contains("monumental")].index

df["Local"][monumental] = "crematório monumental bom pastor"

In [ ]:
limeira = df["Local"].loc[df["Local"].str.contains("limeira")].index

df["Local"][limeira] = "cem. mun de limeira sp"

In [ ]:
conchal = df["Local"].loc[df["Local"].str.contains("conchal")].index

df["Local"][conchal] = "cem. mun de conchal sp"

In [ ]:
rio_claro = df["Local"].loc[df["Local"].str.contains("rio claro")].index

df["Local"][rio_claro] = "cem. mun de rio claro sp"

In [ ]:
df["Local"].value_counts()

cem. mun de araras sp                                                                                    9439
pq dos eucaliptos araras sp                                                                               393
crematório monumental bom pastor                                                                           25
c.                                                                                                         22
cem. mun de rio claro sp                                                                                   20
cem. mun de limeira sp                                                                                     16
cem. mun de conchal sp                                                                                      9
cemiterio municipal.                                                                                        9
cem. mun de leme sp                                                                                         9
crematório

In [ ]:
df.columns

Index(['Nome', 'Idade', 'DataFal', 'DataSep', 'Local', 'Horário'], dtype='object')

In [ ]:
df["Nome"] = df["Nome"].apply(lambda x: x.title())
df["Local"] = df["Local"].apply(lambda x: x.title())

In [ ]:
df.head()

,Nome,Idade,DataFal,DataSep,Local,Horário
0,Gilson De Jesus Tavares,40,31-01-2012,31-01-2012,Cem. Mun De Araras Sp,15:30
1,Luzia De Lourdes Silveira Dias,66,31-01-2012,01-02-2012,Cem. Mun De Araras Sp,15:30
2,Luci Terezinha Martim De Paula,53,30-01-2012,30-01-2012,Cem. Mun De Araras Sp,15:30
3,Valter Sorrantino,47,30-01-2012,31-01-2012,Cem. Mun De Araras Sp,15:30
4,Antonio Vieira Dos Santos,55,30-01-2012,31-01-2012,Cem. Mun De Araras Sp,15:30


In [ ]:
df.shape

(10047, 6)

In [ ]:
df.to_excel("óbitos.xlsx", index=False)